In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=ae6f13c809a2f7e1a92b6a98234069619518ac0a6150a64282ab47a3041ba3e0
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 160.2 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [2]:
!pip install adversarial-robustness-toolbox

     |████████████████████████████████| 491kB 3.0MB/s 
     |████████████████████████████████| 7.1MB 6.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [0]:
from __future__ import print_function
import torch
import torchvision
import torch.nn as nn
import torch.quantization
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.quantization import QuantStub, DeQuantStub
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
from torchvision import datasets, transforms
import numpy as np
from sklearn import metrics
import math
import os
import matplotlib.pyplot as plt

from art.attacks import CarliniL2Method, CarliniLInfMethod, ProjectedGradientDescent
from art.classifiers import PyTorchClassifier
from art.utils import load_cifar10

%matplotlib inline
%config InlineBackend.figure_format='retina'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_cifar10()

x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

train_dataset = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=64)

test_dataset = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=1000)
test_dataloader_single = DataLoader(test_dataset, batch_size=1)

In [0]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.dropout = nn.Dropout(p=0.1)
        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    

def train_classifier(cl, opt, x, y):
    x.to(device)
    y.to(device)
    opt.zero_grad()
    pred = cl(x)
    pred.to(device)
    err = F.nll_loss(F.log_softmax(pred, dim=0), y)
    err.backward()
    opt.step()
    return err, pred

def test_model(cl,test_loader): 
    correct = 0
    with torch.no_grad():
        cl.eval()
        for data, target in test_loader:
            output = cl(data.to(device))
            pred = output.data.max(1, keepdim=True)[1].to("cpu")
            target = np.argmax(target, axis=1)
            correct += pred.eq(target.data.view_as(pred)).sum()
        acc_test = float(correct.numpy() / len(test_loader.dataset))
        
    return acc_test

def print_size_of_model(model):
  torch.save(model.state_dict(), 'temp.p')
  print('Size (KB):', os.path.getsize("temp.p")/1e3)
  os.remove('temp.p')

In [0]:
def make_classifier(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
    # Make a classifier wrapper!
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(min_, max_),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 32, 32),
        nb_classes=10,
    )
    return classifier



def adversarial_training(model, device, data, epsilon, alpha):
    dim = data.size()
    batch = torch.Tensor()
    fgsm = np.random.choice([0, 1], size=dim[0], p=[.5, .5])
    for image, is_fgsm in zip(data, fgsm):
        # VIP Reshape image for model compatibility
        image = image.view(1, dim[1], dim[2], dim[3])
        image, batch = image.to(device), batch.to(device)
        if is_fgsm:
            classifier = make_classifier(model)
            adv_crafter = ProjectedGradientDescent(classifier, norm=np.inf, eps=epsilon, eps_step=alpha, max_iter=7)
            perturbed_image = adv_crafter.generate(x=image.cpu().detach().numpy())
            batch = torch.cat((batch, torch.Tensor(perturbed_image).to(device)), dim=0)
        else:
            batch = torch.cat((batch, image), dim=0)
    return batch

In [19]:
%cd '/content/drive/My Drive/Colab Notebooks/models'

/content/drive/My Drive/Colab Notebooks/models


In [12]:
file = 'quant.pth'

data = np.vstack((x_train, x_test))
cifar10_std = data.std()
eps = (4 / 255.) / cifar10_std
alpha = (2 / 255.) / cifar10_std
print(eps, alpha)

0.06236745581885411 0.031183727909427056


In [0]:
EPOCHS = 20

qat_model = Model().to(device)
qat_model.config = torch.quantization.get_default_qat_qconfig('fbgemm')
# prepare_qat performs the “fake quantization”, preparing the model for quantization-aware training
torch.quantization.prepare_qat(qat_model, inplace=True)

criterion = nn.CrossEntropyLoss()
cl_opt = optim.Adam(qat_model.parameters(), lr=.001, weight_decay=0)

for e in range(EPOCHS):
  qat_model.train()
  for batch_idx, (data, target) in enumerate(train_dataloader):
    target = np.argmax(target, axis=1)  # transform from one-hot to int
    new_batch = adversarial_training(qat_model, device, data, epsilon=eps, alpha=alpha)
    error, pred = train_classifier(qat_model.to(device), cl_opt, new_batch.to(device), target.to(device))  
  # Check accuracy after each epoch
  quantized_model = torch.quantization.convert(qat_model.eval(), inplace=False)
  acc = test_model(quantized_model, test_dataloader)
  print("Epoch ", e+1, ": Test accuracy: ", 100*acc, "%")
  torch.save(qat_model.state_dict(), '{name}_{epoch}'.format(name=file, epoch=e+1))

print("Performance in the trained model")
print('Test Accuracy: ', test_model(quantized_model, test_dataloader)*100, '%')
print("FINISHED!!!")

/usr/local/lib/python3.6/dist-packages/torch/quantization/quantize.py:143: UserWarning: None of the submodule got qconfig applied. Make sure you passed correct configuration through `qconfig_dict` or by assigning the `.qconfig` attribute directly on submodules
  warnings.warn("None of the submodule got qconfig applied. Make sure you "
